In [2]:
# load style
from ROOT import gROOT

path = '../../'
# path = '/eos/user/y/youyou/'
gROOT.SetMacroPath(path+"atlasstyle/")
gROOT.LoadMacro("AtlasLabels.C")
gROOT.LoadMacro("AtlasStyle.C")
gROOT.LoadMacro("AtlasUtils.C")
from ROOT import SetAtlasStyle, ATLASLabel, myText, myMarkerText
SetAtlasStyle()

Welcome to JupyROOT 6.26/02

Applying ATLAS style settings...



In [3]:
# load lib
import ROOT
import os, sys
import numpy as np
from array import array
# import pandas as pd

In [4]:
# load files

# file for standard model
sm_file = path + 'EFT_MINI_TREES/SM.root'
# standard model tree
sm_data = ROOT.TFile(sm_file,'READ')
sm_tree = sm_data.vbswy_tree

eft_path = path + 'EFT_MINI_TREES/'
files = [(eft_path+i) for i in os.listdir(eft_path)]
files.sort()
files.remove(sm_file)
int_files = [i for i in files if 'INT' in i]
quad_files = [i for i in files if 'QUAD' in i]
# all file from john for testing
# all_files = [i for i in files if not (i in int_files) and not(i in quad_files)]

# I have combined the _int.root, _quad.root and the sm.root files to _all.root
# all_files = [('./all/'+i) for i in os.listdir('./all/')]

operators = [i.split('/')[-1].split('_')[0] for i in int_files]
operators.index('T7')

12

In [5]:
# list of branches we interested in 
names = [g.GetName() for g in sm_tree.GetListOfBranches()]
truth_branch = [i for i in names if 'truth' in i]
truth_branch = list(dict.fromkeys(truth_branch))

is_reconst = lambda i: not (('truth' in i) or ('weight' in i) or ('pass_baseline' in i))
reconst_branch = [i for i in names if is_reconst(i)]
reconst_branch = list(dict.fromkeys(reconst_branch))

In [ ]:
# function of method 1
def plot_branch(operator, branch, set_log=True, set_log_x=False, isTruth=True, isPDF=False, default_nbins=45):
    
    # load files
    i = operators.index(operator)
    int_data = ROOT.TFile(int_files[i],'READ')
    int_tree = int_data.vbswy_tree
    quad_data = ROOT.TFile(quad_files[i], 'READ')
    quad_tree = quad_data.vbswy_tree
    min_x = min(int_tree.GetMinimum(branch), quad_tree.GetMinimum(branch), sm_tree.GetMinimum(branch), 0)
    max_x = max(int_tree.GetMaximum(branch), quad_tree.GetMaximum(branch), sm_tree.GetMaximum(branch))+0.5

    def generate_output_path():
        folder = 'reconstructed/'
        if isTruth:
            folder = 'truth/'
        format = 'png'
        if isPDF:
            format = 'pdf'
        folder = './output_'+format+'/'+folder
        if not os.path.exists(folder+operator):
            os.mkdir(folder+operator)
        return (folder+operator+'/'+branch+'.'+format)

    # start plot
    can = ROOT.TCanvas('can1', operator+'/'+branch,800,600)
    can.cd()
    
    # upper part
    pad1 = ROOT.TPad (" pad1 "," pad1 " ,0,0,1,1)
    pad1.Draw()
    pad1.SetLogy ( set_log )
    pad1.SetLogx(set_log_x)
    pad1.cd()
    
    # set bins for hist
    nbins = default_nbins
    if set_log:
        if set_log_x:
            nbins = 40
            if max_x > 10**6:
                nbins = 60
            elif max_x > 10**5:
                nbins = 50
        bins = np.linspace(1.2,1.2,nbins).cumprod()
        bins = np.insert(bins, 0, 0)
        nbins = (len(bins)-1)
        divider = bins[-1] / max_x
        bins = array('d',bins/divider)
        h_sm = ROOT.TH1F("h_sm", "sm;;", nbins, bins)
        h1 = ROOT.TH1F("h1", "c1;;", nbins, bins)
        h2 = ROOT.TH1F("h2", "c2;;", nbins, bins)
        h3 = ROOT.TH1F("h1", "c3;;", nbins, bins)
    else:
        h_sm = ROOT.TH1F("h_sm", "sm;;", nbins, min_x, max_x)
        h1 = ROOT.TH1F("h1", "c1;;", nbins, min_x, max_x)
        h2 = ROOT.TH1F("h2", "c2;;", nbins, min_x, max_x)
        h3 = ROOT.TH1F("h3", "c3;;", nbins, min_x, max_x)
    
    c1, c2, c3 = 1,10,100
    if i >= 7:
        c1, c2, c3 = 0.1,1,10

    if isTruth:
        for i in range(sm_tree.GetEntries()):
            sm_tree.GetEntry(i)
            h_sm.Fill(getattr(sm_tree, branch), sm_tree.weight)
            h1.Fill(getattr(sm_tree, branch), sm_tree.weight)
            h2.Fill(getattr(sm_tree, branch), sm_tree.weight)
            h3.Fill(getattr(sm_tree, branch), sm_tree.weight)
        for i in range(int_tree.GetEntries()):
            int_tree.GetEntry(i)
            h1.Fill(getattr(int_tree, branch), int_tree.weight*c1)
            h2.Fill(getattr(int_tree, branch), int_tree.weight*c2)
            h3.Fill(getattr(int_tree, branch), int_tree.weight*c3)
        for i in range(quad_tree.GetEntries()):
            quad_tree.GetEntry(i)
            h1.Fill(getattr(quad_tree, branch), quad_tree.weight*c1**2)
            h2.Fill(getattr(quad_tree, branch), quad_tree.weight*c2**2)
            h3.Fill(getattr(quad_tree, branch), quad_tree.weight*c3**2)
    else:
        for i in range(sm_tree.GetEntries()):
            sm_tree.GetEntry(i)
            tmp = int(sm_tree.pass_baseline_e) | int(sm_tree.pass_baseline_mu)
            if tmp:
                h_sm.Fill(getattr(sm_tree, branch), sm_tree.weight)
                h1.Fill(getattr(sm_tree, branch), sm_tree.weight)
                h2.Fill(getattr(sm_tree, branch), sm_tree.weight)
                h3.Fill(getattr(sm_tree, branch), sm_tree.weight)
        for i in range(int_tree.GetEntries()):
            int_tree.GetEntry(i)
            tmp = int(int_tree.pass_baseline_e) | int(int_tree.pass_baseline_mu)
            if tmp:
                h1.Fill(getattr(int_tree, branch), int_tree.weight*c1)
                h2.Fill(getattr(int_tree, branch), int_tree.weight*c2)
                h3.Fill(getattr(int_tree, branch), int_tree.weight*c3)
        for i in range(quad_tree.GetEntries()):
            quad_tree.GetEntry(i)
            tmp = int(quad_tree.pass_baseline_e) | int(quad_tree.pass_baseline_mu)
            if tmp:
                h1.Fill(getattr(quad_tree, branch), quad_tree.weight*c1**2)   
                h2.Fill(getattr(quad_tree, branch), quad_tree.weight*c2**2)
                h3.Fill(getattr(quad_tree, branch), quad_tree.weight*c3**2)
    h_sm.SetMarkerStyle(53) 
    h1.SetMarkerColorAlpha(ROOT.kBlue, 0.75)
    h2.SetMarkerColorAlpha(ROOT.kRed+1, 0.75)
    h3.SetMarkerColorAlpha(ROOT.kOrange, 1)
    hs = ROOT.THStack ("hs"," THStack ")
    hs.Add (h3)
    hs.Add (h2)
    hs.Add (h1)
    hs.Add (h_sm)
    hs.Draw ("NOSTACK")

    if set_log:
        n = nbins
        while (n >= 1) and (h3.GetBinContent(n) <= 0.5):
            n -= 1
        max_x = bins[n]
        hs.GetXaxis().SetRangeUser(0, max_x) 
        hs.SetMinimum(0.5)

    legend = ROOT.TLegend (0.65, 0.75,0.93,0.93)
    legend.AddEntry ( h_sm ,'sm.root')
    legend.AddEntry ( h1 ,operator +' with coupling='+str(c1))
    legend.AddEntry ( h2 ,operator +' with coupling='+str(c2))
    legend.AddEntry ( h3 ,operator +' with coupling='+str(c3))
    legend.Draw (" same ")

    hs.GetYaxis().SetTitle("Number")
    hs.GetYaxis().SetTitleOffset (0.8)
    hs.GetYaxis().SetLabelSize(0.02)
    pad1.SetBottomMargin (0.25)
    pad1.SetLeftMargin(0.1)
    
    # lower part
    pad2 = ROOT.TPad (" pad2 "," pad2 " ,0 ,0 ,1 ,0.25)
    pad2.Draw()
    pad2.cd()
    
    h1_low = h1-h_sm
    h1_low.Divide(h_sm)
    h1_low.Draw()

    h2_low = h2-h_sm
    h2_low.Divide(h_sm)
    h2_low.Draw('same')
    h3_low = h3-h_sm
    h3_low.Divide(h_sm)
    h3_low.Draw('same')

    h1_low.GetYaxis().SetRangeUser(-1,1)
    if set_log:
        h1_low.GetXaxis().SetRangeUser(0, max_x) 
    h1_low.GetYaxis().SetNdivisions (207)
    '''markers = []
    for i in range(nbins):
        if (h1_low.GetBinContent(i) > 1):
            markers += [ROOT.TMarker(h1_low.GetBinCenter(i), 1,22)]
        if (h1_low.GetBinContent(i) < -1):
            markers += [ROOT.TMarker(h1_low.GetBinCenter(i), -1,23)] 
    for j in markers:
        j.Draw('same')'''

    line = ROOT.TLine (min_x, 0, max_x,0)#*bins[1], 0)
    line.SetLineColor (1)
    line.SetLineWidth (2)
    line.Draw (" same ")
    
    # set format
    if (branch.split('_')[-1]=='e'):
        h1_low.GetXaxis().SetTitle(branch+'[GeV]')
    elif (branch.split('_')[-1] =='pt'):
        h1_low.GetXaxis().SetTitle(branch+'[GeV/c]')
    elif (branch.split('_')[-1] in ['mt', 'm']):
        h1_low.GetXaxis().SetTitle(branch+'[GeV/c^{2}]')
    else:
        h1_low.GetXaxis().SetTitle(branch)
    h1_low.GetYaxis().SetTitle ("ALL/SM-1  ")
    h1_low.GetXaxis().SetTitleSize (0.25)
    h1_low.GetYaxis().SetTitleSize (0.14)
    h1_low.GetXaxis().SetLabelSize(0.15)
    h1_low.GetYaxis().SetLabelSize(0.14)
    h1_low.GetYaxis().SetTitleOffset (0.27)
    h1_low.GetXaxis().SetTitleOffset (0.7)

    pad2.SetTopMargin (0)
    pad2.SetLeftMargin(0.1)
    pad2.SetBottomMargin (0.4)
    pad2.SetLogx(set_log_x)

    can.Draw()
    can.Update()
    can.Print('test.png')
    can.SaveAs(generate_output_path())
    can.Clear()
    
    int_data.Close()
    quad_data.Close()

In [ ]:
#plot_branch('M2', 'j1_truth_eta', set_log=1, set_log_x=1, isTruth=True)

In [ ]:
for o in operators:
    for t in truth_branch:
        boo = t.split('_')[-1] in ['e','pt','m','mt']
        plot_branch(o, t, set_log=boo, set_log_x=0, isTruth=1, isPDF=1)
    for r in reconst_branch:
        boo = r.split('_')[-1] in ['e','pt','m','mt']
        boo_x = (r.split('_')[0] in ['lep', 'w', 'wy']) and boo
        plot_branch(o, r, set_log=boo, set_log_x=boo_x, isTruth=0, isPDF=1)
    print(o+' completed!')

In [ ]:
'''from PIL import Image

arr = ['p1.png', 'p2.png', 'p3.png', 'p4.png']
toImage = Image.new('RGBA',(400,400))
for i in range(4):
    fromImge = Image.open(arr[i])
    # loc = ((i % 2) * 200, (int(i/2) * 200))
    loc = ((int(i/2) * 200), (i % 2) * 200)
    print(loc)
    toImage.paste(fromImge, loc)

toImage.save('merged.png')'''

In [10]:
from PIL import Image


t_digrams = []
r_digrams = []
t_path = './output_pdf/truth/'
r_path = './output_pdf/reconstructed/'

for o in operators:
    digrams = []
    for t in truth_branch:
        digrams += [t_path+o+'/'+t+'.pdf']
    t_digrams += [digrams]
    digrams = []
    for r in reconst_branch:
        digrams += [r_path+o+'/'+r+'.pdf']
    r_digrams += [digrams]

In [13]:

arr = t_digrams[0]
toImage = Image.new('RGBA',(3200,4200)) # 4800 for recon
for i in range(len(arr)):
    fromImge = Image.open(arr[i])
    # loc = ((i % 2) * 200, (int(i/2) * 200))
    loc = ((i%2)*800, (6-i//2)*600)
    print(loc)
    toImage.paste(fromImge, loc)

toImage.save(t_path+'merged.pdf')
        

UnidentifiedImageError: cannot identify image file './output_pdf/truth/M0/jj_truth_drap.pdf'